In [ ]:
import json
import pandas as pd

entity_data = pd.read_csv("asset/product-list.csv")
list(entity_data['product'])

entity_data_json = json.dumps({
    "product":list(entity_data['product'])
    },indent=4)
with open("asset/entity.json","w") as file:
    file.write(entity_data_json)

In [ ]:
import json

with open ("asset/entity.json") as file:
    data = json.load(file)

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner)
else:
    ner = nlp.create_pipe('ner')

In [ ]:
from tracemalloc import start
import pandas as pd

ner_data = pd.read_csv("asset/Product-NER.csv")
data , char_start, char_end = list (ner_data.question),list(ner_data.start),list(ner_data.end)
for idx, elem in enumerate(data):
    print("-"+elem[char_start[idx]:char_end[idx]]+"-")

CNER Training data Format:
```
[('training sentence',{'entities': [(start-char, end-char, 'entity-code')]})]
```

In [ ]:
CNER_training_data = []
for sentence, start_char, end_char in zip(data, char_start, char_end):
    temp = (
        sentence,{
            "entities":[
                (start_char,end_char,"PRODUCT")
            ]
        }
    )
    CNER_training_data.append(temp)


In [ ]:
CNER_training_data_json = json.dumps({
    "data":CNER_training_data
},indent=4)
with open("asset/product_ner_training_data.json","w") as file:
    file.write(CNER_training_data_json)

In [ ]:
def json_to_CNER_training_data(json_data):
    entities = []
    for elem in json_data:
        temp =[]
        for loc in elem[1]['entities']:
            temp.append(tuple(loc))
        entities.append({
            "entities":temp
        })
    refined_data = []
    for idx,elem in enumerate(json_data):
        refined_data.append(
            tuple([elem[0],entities[idx]])
        )
    return refined_data

In [ ]:
with open("asset/product_ner_training_data.json","r") as file:
    file = json.load(file)
CNER_training_data=  json_to_CNER_training_data(file['data'])
CNER_training_data

In [ ]:
import random
import matplotlib.pyplot as plt

def train_spacy(train_data, iterations):
    nlp = spacy.blank('en')
    if 'ner' not in nlp.pipe_names:
        # ner = nlp.create_pipe('ner')
        nlp.add_pipe("ner", last=True)
       
    # Add labels
    for _, annotations in train_data:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # Disable all pipes other than 'ner' during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        
        train_loss = []
        
        # Go through the training data N times
        for itn in range(iterations):
            print("Starting iteration " + str(itn))

            random.shuffle(train_data)

            losses = {}
            misalligned_count = 0

            for batch in spacy.util.minibatch(train_data, size=2):
                for text, annotations in batch:    
                    try:
                        doc = nlp.make_doc(text)
                        example = spacy.training.Example.from_dict(doc, annotations)
                        nlp.update(
                            [example],
                            drop=0.2,
                            sgd=optimizer,
                            losses=losses)

                    except ValueError as e:
                        misalligned_count += 1
                        print(f'Ignoring misaligned entity...\n{(text,annotations)}')
                        pass

                train_loss.append(losses.get('ner'))
                print(f'losses (iteration {itn}): {losses}')
            
        # Visualizing the loss
        plt.figure(figsize=(10,6))
        plt.plot([*range(len(train_loss))], train_loss, color = 'magenta')
        plt.title('Loss at every iteration')
        plt.xlabel('Iteration Number')
        plt.ylabel('Loss')
        plt.show()
            
    return nlp

In [ ]:
product_nlp = train_spacy(train_data=CNER_training_data,iterations=20)

In [70]:
import pickle
pickle.dump(product_nlp, open('models/product_nlp.pkl', "wb"))

In [71]:
nlp=pickle.load(open("models/product_nlp.pkl", "rb"))

In [72]:
from IPython.core.display import display, HTML
from spacy import displacy
def extract_product(user_input, visualize = False):
    product_NLP = product_nlp
    doc = product_nlp(user_input)

    extracted_entities = []
    for ent in doc.ents:
        extracted_entities.append((ent.text, ent.start_char, ent.end_char, ent.label_))

    if visualize == True:
        colors = {"PRODUCT": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}
        options = {"ents": ["PRODUCT"], "colors": colors}
        html = displacy.render(doc, style = 'ent', options = options)
        display(HTML(html));
    return extracted_entities

/tmp/ipykernel_1362/3018919082.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [65]:
from IPython.core.display import display, HTML
from spacy import displacy
def extract_product2(user_input, visualize = False):
    product_NLP = nlp
    doc = nlp(user_input)

    extracted_entities = []
    for ent in doc.ents:
        extracted_entities.append((ent.text, ent.start_char, ent.end_char, ent.label_))

    if visualize == True:
        colors = {"PRODUCT": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}
        options = {"ents": ["PRODUCT"], "colors": colors}
        html = displacy.render(doc, style = 'ent', options = options)
        display(HTML(html));
    return extracted_entities

/tmp/ipykernel_1362/1154706093.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [97]:
x = extract_product2("when is my active subscription",visualize=True)

<IPython.core.display.HTML object>

In [84]:
text = "when is my d365 expiring"
newtext = text[:x[0][1]] + x[0][3]+" "+ text[x[0][1]-1+len(x[0][3])-1:]
newtext

'when is my PRODUCT expiring'